# Audio classification

We have a set of voice recordings from different people. We need to get these classified according to the speaker’s gender. We ask performers to listen to the recordings and decide whether it is a man or a woman speaking.

To get acquainted with Toloka tools for free, you can use the promo code **TOLOKAKIT1** on $20 on your [profile page](https://toloka.yandex.com/requester/profile) after registration. 

Prepare environment and import all we'll need.

In [ ]:
!pip install toloka-kit==0.1.11
!pip install crowd-kit==0.0.5
!pip install pandas

import datetime
import sys
import time
import logging

import pandas
import numpy as np

import toloka.client as toloka
import toloka.client.project.template_builder as tb
from crowdkit.aggregation import DawidSkene

logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

Сreate toloka-client instance. All api calls will go through it. More about OAuth token in our [Learn the basics example](https://github.com/Toloka/toloka-kit/tree/main/examples/0.getting_started/0.learn_the_basics) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/0.getting_started/0.learn_the_basics/learn_the_basics.ipynb)

In [ ]:
toloka_client = toloka.TolokaClient(input("Enter your token:"), 'PRODUCTION')  # Or switch to 'SANDBOX'
print(toloka_client.get_requester())

## Create a project
Enter a clear project name and description.
> Note: The project name and description will be visible to the performers.

In [ ]:
project = toloka.Project(
    public_name='Is it a male or female speaker?',
    public_description='Listen to the audio and determine if it is a male or female speaker.',
)

Create task interface.
> Check the [Interface section](https://toloka.ai/knowledgebase/interface) of our Knowledge Base for more tips on interface design.

In [ ]:
audio_viewer = tb.AudioViewV1(
    tb.InputData('path'),
    validation=tb.PlayedConditionV1(hint='you need to listen to the audio'),
)

radio_group_field = tb.ButtonRadioGroupFieldV1(
    tb.OutputData('result'),
    [
        tb.GroupFieldOption('female', 'Female'),
        tb.GroupFieldOption('male', 'Male'),
    ],
    label='Is it a male or female speaker?',
    validation=tb.RequiredConditionV1(),
)

task_width_plugin = tb.TolokaPluginV1(
    layout=tb.TolokaPluginV1.TolokaPluginLayout(
        kind='scroll', 
        task_width=300,
    )
)

hot_keys_plugin = tb.HotkeysPluginV1(
    key_1=tb.SetActionV1(tb.OutputData('result'), 'female'),
    key_2=tb.SetActionV1(tb.OutputData('result'), 'male'),
)

project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ListViewV1([audio_viewer, radio_group_field]),
    plugins=[task_width_plugin, hot_keys_plugin],
)

For performers, our interface will look like this.

<table  align="center">
  <tr><td>
    <img src="./img/tasks_preview.png"
         alt="Task page"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> What the task page can looks like.
  </td></tr>
</table>

Specifications are a description of input data that will be used in a project and the output data that will be collected from the performers.

> Read more about [input and output data specifications](https://yandex.ru/support/toloka-tb/operations/create-specs.html?lang=en) in the Requester’s Guide.

In [ ]:
input_specification = {'path': toloka.project.UrlSpec()}
output_specification = {'result': toloka.project.StringSpec()}

project.task_spec = toloka.project.task_spec.TaskSpec(
    input_spec=input_specification,
    output_spec=output_specification,
    view_spec=project_interface,
)

Write comprehensive instructions.
> Get more tips on [designing instructions](https://toloka.ai/knowledgebase/instruction) in our Knowledge Base.

In [ ]:
project.public_instructions = """Listen to the short audio clip and determine whether it is a male or female speaking."""

Create a project.

In [ ]:
project = toloka_client.create_project(project)

## Preparing data
This example uses [EmoV-DB dataset](https://github.com/numediart/EmoV-DB).

BibTex:
```
@article{adigwe2018emotional,
  title={The emotional voices database: Towards controlling the emotion dimension in voice generation systems},
  author={Adigwe, Adaeze and Tits, No{\'e} and Haddad, Kevin El and Ostadabbas, Sarah and Dutoit, Thierry},
  journal={arXiv preprint arXiv:1806.09514},
  year={2018}
}
```

Let's load this dataset and split it into two parts.

In [36]:
!curl https://tlk.s3.yandex.net/ext_dataset/emov-db-mp3/emov_db.tsv --output dataset.csv

dataset = pandas.read_csv('dataset.csv', sep='\t')
print(dataset)

dataset = dataset.sample(frac=1).reset_index(drop=True)
golden_dataset, main_dataset, _ = np.split(dataset, [20, 120], axis=0)

print(f'\ngolden_dataset - {len(golden_dataset)}')
print(f'\nmain_dataset - {len(main_dataset)}')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1038k  100 1038k    0     0  2740k      0 --:--:-- --:--:-- --:--:-- 2740k
      emotion speaker     sex  \
0      Amused   jenie  female   
1      Amused   jenie  female   
2      Amused   jenie  female   
3      Amused   jenie  female   
4      Amused   jenie  female   
...       ...     ...     ...   
6888  Neutral    josh    male   
6889  Neutral    josh    male   
6890  Neutral    josh    male   
6891  Neutral    josh    male   
6892  Neutral    josh    male   

                                                 phrase  \
0                      I had forgotten their existence.   
1     His outstretched arm dropped to his side, and ...   
2                    He spat it out like so much venom.   
3     A big canvas tent was the first thing to come ...   
4                   Gad, your letter came just in time.   
...         

## Create the main pool
A pool is a set of paid tasks grouped into task pages. These tasks are sent out for completion at the same time. 

>Note: All tasks within a pool have the same settings (price, quality control, etc.)

Audio classification tasks are normally paid as basic tasks (e.g. binary classification) because these tasks do not take much time. Read more about [pricing principles](https://toloka.ai/knowledgebase/pricing) in our Knowledge Base.

Sets an overlap of 3 to get a more confident final label. To understand [how this rule works](https://yandex.com/support/toloka-requester/concepts/mvote.html), go to the Requester’s Guide.

Let's add language filter so performers who speak English will be invited to complete this task. Then choose Toloka web version and Toloka for mobile clients. These filters will make it possible for performers to complete your task on their computers or mobile devices.

In [ ]:
pool = toloka.Pool(
    project_id=project.id,
    # Give the pool any convenient name. You are the only one who will see it.
    private_name='Is it a male or female speaker',
    may_contain_adult_content=False,
    # Set the price per task page.
    reward_per_assignment=0.01,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    # Overlap. This is the number of users who will complete the same task.
    defaults=toloka.Pool.Defaults(default_overlap_for_new_task_suites=3),
    # Time allowed for completing a task page
    assignment_max_duration_seconds=1200,
    filter=(
        (toloka.filter.Languages.in_('EN')) &
        (
            (toloka.filter.ClientType == 'TOLOKA_APP') | 
            (toloka.filter.ClientType == 'BROWSER')
        )
    ),
)

Set up [Quality control](https://yandex.com/support/toloka-requester/concepts/control.html):
  - Ban performers who give incorrect responses to control tasks. Since tasks such as these have an answer that can be used as ground truth, we can use standard quality control rules like golden sets.

Read more about [quality control principles](https://toloka.ai/knowledgebase/quality-control) in our Knowledge Base or check out [control tasks settings](https://yandex.com/support/toloka-requester/concepts/goldenset.html) in the Requester’s Guide.

In [ ]:
pool.quality_control.add_action(
    collector=toloka.collectors.GoldenSet(history_size=10),
    conditions=[
        toloka.conditions.GoldenSetCorrectAnswersRate <= 80.0,
        toloka.conditions.GoldenSetAnswersCount >= 1
    ],
    action=toloka.actions.RestrictionV2(
        scope='POOL',
        duration=3,
        duration_unit='DAYS',
        private_comment='bad quality'
    )
)

Specify	the number of tasks per page. We recommend putting as many tasks on one page as a performer can complete in 1 to 5 minutes. That way, performers are less likely to get tired, and they won’t lose a significant amount of data if a technical issue occurs.

To learn more about [grouping tasks](https://yandex.com/support/search-results/?service=toloka-requester&query=smart+mixing) into suites, read the Requester’s Guide. 

In [ ]:
pool.set_mixer_config(
    real_tasks_count=4,
    golden_tasks_count=1,
)

Create pool

In [ ]:
pool = toloka_client.create_pool(pool)

## Preparing and uploading tasks

> Note: Control tasks are tasks that already contain the correct response. They are used for checking the quality of responses from performers. The performer's response is compared to the response you provided. If they match, it means the performer answered correctly.`

In [ ]:
golden_tasks = [
    toloka.task.Task(
        pool_id=pool.id,
        input_values={'path': row['url']},
        known_solutions = [
            toloka.task.BaseTask.KnownSolution(
                output_values={'result': row['sex']}
            )
        ],
        infinite_overlap=True,
    )
    for _, row in golden_dataset.iterrows()
]
tasks = [
    toloka.task.Task(
        pool_id=pool.id,
        input_values={'path': row['url']},
    )
    for _, row in main_dataset.iterrows()
]
created_tasks = toloka_client.create_tasks(golden_tasks + tasks, allow_defaults=True)
print(len(created_tasks.items))

Start the pool.

**Important.** Remember that real Toloka performers will complete the tasks.
Double check that everything is correct
with your project configuration before you start the pool

In [ ]:
pool = toloka_client.open_pool(pool.id)
print(pool.status)

## Receiving responses

Wait until the pool is completed.

In [ ]:
pool_id = pool.id

def wait_pool_for_close(pool_id, minutes_to_wait=1):
    sleep_time = 60 * minutes_to_wait
    pool = toloka_client.get_pool(pool_id)
    while not pool.is_closed():
        op = toloka_client.get_analytics([toloka.analytics_request.CompletionPercentagePoolAnalytics(subject_id=pool.id)])
        op = toloka_client.wait_operation(op)
        percentage = op.details['value'][0]['result']['value']
        print(
            f'   {datetime.datetime.now().strftime("%H:%M:%S")}\t'
            f'Pool {pool.id} - {percentage}%'
        )
        time.sleep(sleep_time)
        pool = toloka_client.get_pool(pool.id)
    print('Pool was closed.')

wait_pool_for_close(pool_id)

Get responses

When all the tasks are completed, look at the responses from performers.

In [ ]:
answers = []

for assignment in toloka_client.get_assignments(pool_id=pool_id, status='ACCEPTED'):
    for task, solution in zip(assignment.tasks, assignment.solutions):
        if not task.known_solutions:
            answers.append([task.input_values['path'], solution.output_values['result'], assignment.user_id])

print(f'answers count: {len(answers)}')

Aggregation results using the Dawid-Skene model. We use this aggregation model because our questions are of comparable difficulty, and we don't have many control tasks.

Read more about the [Dawid-Skene model](https://yandex.com/support/toloka-requester/concepts/result-aggregation.html#aggr__dawid-skene) in the Requester’s Guide or get at an overview of different [aggregation models](https://toloka.ai/knowledgebase/aggregation) our Knowledge Base.

More aggregation models in [Crowd-Kit](https://github.com/Toloka/crowd-kit#crowd-kit-computational-quality-control-for-crowdsourcing).

In [ ]:
# Prepare dataframe
answers_df = pandas.DataFrame(answers, columns=['task', 'label', 'performer'])
# Run aggregation
predicted_answers = DawidSkene(n_iter=20).fit_predict(answers_df)

print(predicted_answers)